# HowTo -Mask- From RGB images to binary masks

In this notebook you will learn:

1. how to connect to a ROMI database
2. how to use our algorithms to detect the plant in the image and create a binary mask
3. how to visualize the obtained masks

This notebook **assume** that you have:
- declared the `ROMI_DB` environment variable as the path to the database directory to use
- processed the test dataset with the _geometric pipeline,_ so we can access the fileset containing the data we want to start with...

Remember, the aim of this notebook is to show you how it works "under the hood".
This is not how you should process your data, that is done thanks to the `romi_run_task` CLI tool.

In [ ]:
%matplotlib inline

In [ ]:
import os

import ipywidgets as widgets
import matplotlib.pyplot as plt
from plant3dvision.proc2d import linear, excess_green, dilation
from plant3dvision.visu import plotly_image_carousel
from plantdb import FSDB
from plantdb.io import read_image

## Connect to the database & get the initial data

If you did not declare a `ROMI_DB` environment variable, you can do it by uncommenting the next cell and setting it to the right value.

In [ ]:
# os.environ['ROMI_DB'] = "/path/to/test/data"

### Connect to the database

In [ ]:
db = FSDB(os.environ['ROMI_DB'])  # requires definition of this environment variable!
db.connect()

Once you are connected to the database, you can list the available scan *dataset* with `db.list_scans()`.

### Select a dataset

We now select a dataset (with the `Dropdown` widget) for the demo:

In [ ]:
scan_name = widgets.Dropdown(options=db.list_scans(), value=db.list_scans()[0], description='Dataset:')
display(scan_name)

In [ ]:
scan = db.get_scan(scan_name.value)

If you did not process this dataset yet, from the `plant3dvision` root directory, you can do it with:
```
romi_run_task AnglesAndInternodes $ROMI_DB/<selected_dataset> --config plant-3d-vision/configs/geom_pipe_real.toml
```

To list the available *filesets* in this *scan dataset*:

In [ ]:
scan.list_filesets()

### Get the RGB images fileset

The RGB images resulting from an _Undistorted_ tasks are to be found in the 'Undistorted*' fileset.

In [ ]:
img_fs = scan.get_filesets(query={"task_name": "Undistorted"})[0]
print(img_fs.path().stem)

Once you have access to the 'images' fileset, you may access the RGB images as follows:

In [ ]:
img_files = img_fs.get_files(query={"channel": "rgb"})

In [ ]:
print(f"This fileset contains {len(img_files)} files (matching the `query`).")

### Visualize the RGB images fileset

It is possible to visualize the set of RGB images using our `plotly_image_carousel` method.

In [ ]:
fig = plotly_image_carousel(img_files, title=scan_name.value)

In [ ]:
fig.show()

## Extract a binary mask using a _linear_ filter

More details about the _linear_ filter can be accessed on our [technical documentation](https://docs.romi-project.eu/plant_imager/explanations/task_masks/#method-linear).

**This is the default method** used in the geometric workflow.

Using the `read_image` method from `plantdb.io`, you can load an image from a database `File`:

In [ ]:
img = read_image(img_files[0])

We now apply the `linear` filter algorithm to the loaded image as follows:

In [ ]:
filter_img = linear(img, coefs=[0.1, 1., 0.1])

Then we binarize the grayscale image with a _high-pass threshold_:

In [ ]:
threshold = 0.3
mask = filter_img > threshold  # convert to binary mask using threshold

This binary image is then dilated as follows:

In [ ]:
radius = 2
dilated_mask = dilation(mask, radius)  # apply a dilation to binary mask

Finally, we can visualize the original image and the result of each successive step with:

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(8, 7))
axes[0, 0].imshow(img)
axes[0, 0].set_title("Original image")
axes[0, 1].imshow(filter_img, cmap='gray')
axes[0, 1].set_title("Filtered image (linear)")
axes[1, 0].imshow(mask, cmap='gray')
axes[1, 0].set_title(f"Binary mask image (threshold={threshold})")
axes[1, 1].imshow(dilated_mask, cmap='gray')
axes[1, 1].set_title(f"Dilated binary mask image (radius={radius})")
[ax.set_axis_off() for ax in axes.flatten()]
plt.tight_layout()
plt.show()

Note that we do not need a PERFECT masks of the plant, with the space-carving method we will select voxels that occupy the defined volume, so anything from the background that comes in contact with the plant will be removed.
Sam goes for the "noise" that may be seen around the plant.

## Extract a binary mask using an _excess green_ filter

For comparison purposes, we also present the _excess green_ filter method hereafter.

More details about the _excess green_ filter can be accessed on our [technical documentation](https://docs.romi-project.eu/plant_imager/explanations/task_masks/#method-excess_green).

Using the `read_image` method from `plantdb.io`, you can load an image from a database `File`:

In [ ]:
img = read_image(img_files[0])

We now apply the `excess_green` filter algorithm to the loaded image as follows:

In [ ]:
filter_img = excess_green(img)

Then we binarize the grayscale image with a _high-pass threshold_:

In [ ]:
threshold = 0.3
mask = filter_img > threshold  # convert to binary mask using threshold

This binary image is then dilated as follows:

In [ ]:
radius = 2
dilated_mask = dilation(mask, radius)  # apply a dilation to binary mask

Finally, we can visualize the original image and the result of each successive step with:

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(8, 7))
axes[0, 0].imshow(img)
axes[0, 0].set_title("Original image")
axes[0, 1].imshow(filter_img, cmap='gray')
axes[0, 1].set_title("Filtered image (excess green)")
axes[1, 0].imshow(mask, cmap='gray')
axes[1, 0].set_title(f"Binary mask image (threshold={threshold})")
axes[1, 1].imshow(dilated_mask, cmap='gray')
axes[1, 1].set_title(f"Dilated binary mask image (radius={radius})")
[ax.set_axis_off() for ax in axes.flatten()]
plt.tight_layout()
plt.show()

We may now **disconnect** from the database as we will not need it anymore:

In [ ]:
db.disconnect()